## Bibliotecas

In [1]:
import pandas as pd
import joblib

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.svm import SVR

In [2]:
CSV_FILE = 'EDA/df_normalized.csv'

df = pd.read_csv(CSV_FILE)
display(df.head())

In [3]:
#n² 94%

df = df.drop(
    columns = [
        'educacion_padre', 'educacion_madre',
        'nota1', 'nota2'
    ]
)

df

In [4]:
# n² 94%

# df = df.drop(
#     columns = [
#         'educacion_padre', 'educacion_madre',
#         'nota1', 'nota2', 'es_numerosa'
#     ]
# )

# df

In [5]:
# n² 91

# df = df.drop(
#     columns = [
#         'educacion_padre', 'educacion_madre',
#         'nota1', 'nota2', 'tiempo_estudio'
#     ]
# )

# df

In [6]:
# 90%

# df = df.drop(
#     columns = [
#         'educacion_padre', 'educacion_madre',
#         'nota1', 'nota2'
#     ]
# )

# df

In [7]:
# 92,9%

# df = df.drop(
#     columns = [
#         'educacion_padre', 'educacion_madre',
#         'nota1', 'nota2', 'consumo_alcohol_entre_semana'
#     ]
# )

# df

In [8]:
# # n² 85%

# df = df.drop(
#     columns = [
#         'educacion_media',
#         'nota1', 'nota2',
#     ]
# )

# df

In [9]:
# n² 85%

# df = df.drop(
#     columns = [
#         'educacion_media',
#         'nota1', 'nota2',
#     ]
# )

# df

In [10]:
# # #n² 10%

# df = df.drop(
#     columns = [
#         'educacion_media',
#         'nota1', 'nota2', 'nota3'
#     ]
# )

# df

## Modelo notas_media
Este primer modelo se va a centrar en explorar la variable `notas_media` ya que su correlacion era mayor que las 3 notas por separado

In [11]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_df, test_df = train_test_split(
    df,
    test_size=0.15
)

X_train = train_df.drop(columns=['notas_media'])
Y_train = train_df["notas_media"]
X_test = test_df.drop(columns=['notas_media'])
Y_test = test_df["notas_media"]



# display(X_train)
# display(Y_train)
# display(X_test)
# display(X_test)


### Regresión Linear

In [12]:
lr_reg = LinearRegression()

# Validación cruzada
cv_scores = cross_val_score(
    lr_reg,
    X_train, Y_train,
    cv=5, scoring='r2')

lr_reg.fit(X_train, Y_train)
Y_pred = lr_reg.predict(X_test)

# Evaluacion
r2_lr = r2_score(Y_test, Y_pred)
mae_lr = mean_absolute_error(Y_test, Y_pred)
mse_lr = mean_squared_error(Y_test, Y_pred)

In [13]:
print(f'r2_lr = {r2_lr * 100:.2f}%')
print(f'mae_lr = {mae_lr:.3f}')
print(f'mse_lr = {mse_lr:.3f}')

r2_lr = 93.44%
mae_lr = 0.043
mse_lr = 0.003


In [14]:
# Crear y entrenar el modelo
model = GradientBoostingRegressor(n_estimators=100)
model.fit(X_train, Y_train)

# Predecir
Y_pred_gb = model.predict(X_test)

# Evaluación
r2_gb = r2_score(Y_test, Y_pred_gb)
mae_gb = mean_absolute_error(Y_test, Y_pred_gb)
mse_gb = mean_squared_error(Y_test, Y_pred_gb)

# Mostrar las métricas
display(r2_gb)
display(mae_gb)
display(mse_gb)

0.9123078077782611

np.float64(0.04873151057764662)

np.float64(0.004128216922123411)

In [15]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)
ridge.fit(X_train, Y_train)

Y_pred_ridge = model.predict(X_test)

r2_ridge = r2_score(Y_test, Y_pred_ridge)
mae_ridge = mean_absolute_error(Y_test, Y_pred_ridge)
mse_ridge = mean_squared_error(Y_test, Y_pred_ridge)

display(r2_ridge)
display(mae_ridge)
display(mse_ridge)



0.9123078077782611

np.float64(0.04873151057764662)

np.float64(0.004128216922123411)

In [16]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.1)
model.fit(X_train, Y_train)

Y_pred_lasso = model.predict(X_test)

r2_lasso = r2_score(Y_test, Y_pred_lasso)
mae_lasso = mean_absolute_error(Y_test, Y_pred_lasso)
mse_lasso = mean_squared_error(Y_test, Y_pred_lasso)

display(r2_lasso)
display(mae_lasso)
display(mse_lasso)




-0.018766245219787203

np.float64(0.17719919110212334)

np.float64(0.047959663758546836)